In [41]:
pip install caer canaro

In [42]:
import os
import caer
import canaro
import numpy as np
import cv2
import gc

In [43]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [44]:
# creating a dictionary to go throught each folder and get no. of the image in those folders
dict = {}
for char in os.listdir(char_path):
    dict[char] = len(os.listdir(os.path.join(char_path, char)))

# sorting dicttionary in descending order
dict = caer.sort_dict(dict, descending=True)
dict


In [45]:
characters = []
count = 0
for i in dict:
    characters.append(i[0])
    count += 1
    if count>=10:
        break

In [46]:
characters

In [47]:
# creating the training data 
tarin = caer.preprocess_from_dir(char_path, characters, IMG_SIZE=IMG_SIZE, channels=channels, isShuffle=True)

In [48]:
len(tarin)

In [49]:
import matplotlib.pyplot as plt
plt.figure(figsize= (30,30))
plt.imshow(tarin[0][0], cmap='gray')
plt.show()

In [50]:
feature_set, labels= caer.sep_train(tarin, IMG_SIZE=IMG_SIZE)

In [51]:
from tensorflow.keras.utils import to_categorical
#normalize the feature set
feature_set = caer.normalize(feature_set)
labels = to_categorical(labels, len(characters))

In [52]:
x_train, x_val, y_train, y_val = caer.train_val_split(feature_set, labels, val_ratio=0.20)

In [53]:
#del tarin
#del feature_set
#del labels

In [60]:

#gc.collect()

batch_size = 32
epochs = 10

In [61]:
# image data generator
data_gen = canaro.generators.imageDataGenerator()
train_gen = data_gen.flow(x_train, y_train, batch_size=batch_size)

In [62]:
# craeting a model
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels,
                                         output_dim=len(characters), loss='binary_crossentropy',
                                         decay=1e-6, learning_rate=0.001, momentum=0.9, nesterov=True)

In [57]:
model.summary()

In [63]:
from tensorflow.keras.callbacks import LearningRateScheduler
callback_list = [LearningRateScheduler(canaro.lr_schedule)]

In [64]:
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    validation_steps=len(y_val)//batch_size,
                    callbacks=callback_list)

In [73]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_1.jpg'
img = cv2.imread(test_path)
plt.imshow(img, cmap= 'gray')
plt.show()

def prepare(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [74]:
pridictions = model.predict(prepare(img))
pridictions

In [75]:
print(characters[np.argmax(pridictions[0])])